## Sistema de Recomendação de Hoteis

Vamos construir um sistema de recomendação de Hoteis

In [23]:
#importacao das bibliotecas
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re
import random  
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import plotly.graph_objs as go
import cufflinks
pd.options.display.max_columns = 30
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ass_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='solar')

In [5]:
df = pd.read_csv('Seattle_Hotels.csv', encoding="latin-1")

In [6]:
df.head()

,name,address,desc
0,Hilton Garden Seattle Downtown,"1821 Boren Avenue, Seattle Washington 98101 USA","Located on the southern tip of Lake Union, the..."
1,Sheraton Grand Seattle,"1400 6th Avenue, Seattle, Washington 98101 USA","Located in the city's vibrant core, the Sherat..."
2,Crowne Plaza Seattle Downtown,"1113 6th Ave, Seattle, WA 98101","Located in the heart of downtown Seattle, the ..."
3,Kimpton Hotel Monaco Seattle,"1101 4th Ave, Seattle, WA98101",What?s near our hotel downtown Seattle locatio...
4,The Westin Seattle,"1900 5th Avenue, Seattle, Washington 98101 USA",Situated amid incredible shopping and iconic a...


In [7]:
print('No dataset,', len(df), 'hoteis foram encontrados') #qtde de hoteis

No dataset, 152 hoteis foram encontrados


In [8]:
def print_description(index):
    example = df[df.index == index][['desc', 'name']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Name:', example[1])

In [9]:
print_description(5)

More than just a hotel, The Paramount Hotel Seattle summons the feel of cozy, elegant and luxurious private residence with friendly hosts, the most comfortable beds in Seattle and an award-winning Asian restaurant right downstairs. Youll fall in love with the simple luxury and charm of our boutique hotel, from the warm, inviting wood finishes, comfy seating areas and fireplace in the lobby to the classically decorated rooms and suites appointed with a dash of urban flair. Our central downtown location puts you in good company: one block from the Washington State Convention Center and walking distance to Pike Place Market, the Seattle Art Museum and diverse shopping, cafes and restaurants. There are many reasons The Paramount Hotel is annually ranked among the top five hotels in downtown Seattle, Washington. Why not come find out yours? 
Name: The Paramount Hotel Seattle


In [10]:
print_description(50)

Welcome to the Crowne Plaza Seattle Airport Located across the sky bridge from Seattle Tacoma International Airport and adjacent to the train station, where the light rail train leaves every 12 minutes to downtown Seattle. Take the train to visit the Space Needle, Pike Place Market, Westlake Shopping Center, Nordstroms flag ship store, Century Link, Safeco Field, and the University of Washington.Just beyond our front doors are headquarters of Fortune 500 companies such as Boeing, Microsoft, Amazon, Apple, Google, Tableaux, Facebook and Starbucks. Seattle offers everything from professional sports to hiking up Mt. Rainier, leisure guests planning cruises from Seattle love our hotel's "Park, Stay and Go" package. Join us in Reflections Bar and Grill for a Pacific Northwest dining experience featuring local beers and wine with light bites.Crowne Plaza Seattle Airport has over 12,000 square feet of private event space onsite, with 7 meeting rooms, and two Ballrooms, including our 5,000 sq

In [11]:
print_description(10)

Soak up the vibrant scene in the Living Room Bar and get in the mix with our live music and DJ series before heading to a memorable dinner at TRACE. Offering inspired seasonal fare in an award-winning atmosphere, it's a not-to-be-missed culinary experience in downtown Seattle. Work it all off the next morning at FIT®, our state-of-the-art fitness center before wandering out to explore many of the area's nearby attractions, including Pike Place Market, Pioneer Square and the Seattle Art Museum. As always, we've got you covered during your time at W Seattle with our signature Whatever/Whenever® service - your wish is truly our command.
Name: W Seattle


## Visualize Token (vocabulary) Frequency Distribuition Before Removing Stop Words

In [12]:
#funcao de stopwords
%matplotlib inline
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis =0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
    return words_freq[:n]
common_words = get_top_n_words(df['desc'], 20)
df1 = pd.DataFrame(common_words, columns = ['desc', 'count'])
df1.groupby('desc').sum()['count'].sort_values().iplot(kind='barh', yTitle='Count', linecolor='black', title='Top 20 words in hotel description before removing stop words')


In [13]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_frqe = sorted(words_freq, key = lambda x: x[1], reverse = True)
    return words_freq[:n]
common_words = get_top_n_words(df['desc'], 20)
df2 = pd.DataFrame(common_words, columns = ['desc', 'count'])
df2.groupby('desc').sum()['count'].sort_values().iplot(kind='barh', yTitle='count', linecolor = 'black', title='Top 20')

## Bigrama, dois termos antes do stop words

In [14]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0,idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(df['desc'], 20)
df3 = pd.DataFrame(common_words, columns = ['desc', 'count'])
df3.groupby('desc').sum()['count'].sort_values(ascending=False).iplot(kind='bar', yTitle='count', linecolor = 'black', title='TOP 20')

Trigrama, tres termos antes do stop words

In [15]:
def get_top_n_trigram(corpus , n=None):
    vec = CountVectorizer(ngram_range=(3, 3)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
    return words_freq[:n]
common_words = get_top_n_trigram(df['desc'], 20)
df5 = pd.DataFrame(common_words, columns = ['desc', 'count'])
df5.groupby('desc').sum()['count'].sort_values(ascending=False).iplot(kind = 'bar', yTitle = 'Count', linecolor = 'black', title='TOP20')

In [16]:
df['word_count'] = df['desc'].apply(lambda x: len(str(x).split()))

In [17]:
df

,name,address,desc,word_count
0,Hilton Garden Seattle Downtown,"1821 Boren Avenue, Seattle Washington 98101 USA","Located on the southern tip of Lake Union, the...",184
1,Sheraton Grand Seattle,"1400 6th Avenue, Seattle, Washington 98101 USA","Located in the city's vibrant core, the Sherat...",152
2,Crowne Plaza Seattle Downtown,"1113 6th Ave, Seattle, WA 98101","Located in the heart of downtown Seattle, the ...",147
3,Kimpton Hotel Monaco Seattle,"1101 4th Ave, Seattle, WA98101",What?s near our hotel downtown Seattle locatio...,150
4,The Westin Seattle,"1900 5th Avenue, Seattle, Washington 98101 USA",Situated amid incredible shopping and iconic a...,151
...,...,...,...,...
147,The Halcyon Suite Du Jour,"1125 9th Ave W, Seattle, WA 98119","Located in Queen Anne district, The Halcyon Su...",198
148,Vermont Inn,"2721 4th Ave, Seattle, WA 98121",Just a block from the world famous Space Needl...,98
149,Stay Alfred on Wall Street,"2515 4th Ave, Seattle, WA 98121",Stay Alfred on Wall Street resides in the hear...,143
150,Pike's Place Lux Suites by Barsala,"2nd Ave and Stewart St, Seattle, WA 98101",The perfect marriage of heightened convenience...,57


In [18]:
desc_lengths = list(df['word_count'])

print('Number of descriptions:', len(desc_lengths),
      '\nAverage word count', np.average(desc_lengths),
      '\nMinimun word count', min(desc_lengths),
      '\nMaximun word conut', max(desc_lengths))

Number of descriptions: 152 
Average word count 156.94736842105263 
Minimun word count 16 
Maximun word conut 494


In [19]:
%matplotlib inline
df['word_count'].iplot(
    kind='hist',
    bins=50,
    linecolor='black',
    xTitle='word count',
    yTitle='count',
    title='Contagem de Palavras na Descrição dos Hoteis'
)

In [24]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: o texto é uma string

        return: o output será a versão limpa
    """
    text = text.lower() 
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

df['desc_clean'] = df['desc'].apply(clean_text)

In [26]:
def print_description(index):
    example = df[df.index == index][['desc_clean', 'name']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Name:', example[1])
print_description(7)

downtown seattle destination hotel steps everywhere want seattle style motif local welcome libation rooftop lounge views across city touchstones areas sweeping landscape rich arts music culture infuse surroundings newly renovated rooms feel like modern residences natural hardwoods colors inspired pacific northwest youll find regions culinary bounty reflected menus frolik kitchen + cocktails adjoining fifth floor patio join us 5th avenue pike union shopping pike place market waterfront convention center seattle monorail light rail airport stroll away
Name: Motif Seattle


In [27]:
def print_description(index):
    example = df[df.index == index][['desc_clean', 'name']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Name:', example[1])
print_description(2)

located heart downtown seattle awardwinning crowne plaza hotel seattle downtown offers exceptional blend service style comfort youll notice cool comfortable unconventional touches set us apart soon step inside marvel stunning views city lights relaxing new sleep advantage beds enjoy complimentary wireless internet throughout hotel amenities help relax like temple spa sleep tight amenity kits featuring lavender spray lotions help rejuvenate unwind enjoy invigorating workout 24hour fitness center get dining suggestions expert concierge savor sumptuous cuisine regatta bar grille restaurant enjoy happy hour lounge daily 4pm 7pm monthly drink specials come experience emerald city offer us
Name: Crowne Plaza Seattle Downtown


In [28]:
print_description(100)

budget seattle looking something different historic charm home away home atmosphere baroness sure make feel like one family conveniently located first hill proud part virginia mason hospital campus minutes harborview medical center swedish hospital baroness hotel great option short long term medical patient family stays whether visiting areas worldclass medical facilities budget vacation goal ensure wonderful stay guest amenities complimentary internet access two twin one two queen studios mini fridge microwave two twin one queen suites full kitchens laundry facilities available flat screen cable television hbo complimentary local calls ice vending machines located lobby coffee maker hairdryers guestrooms room service available seven days week rhododendron cafe limited wheelchair accessibility guest library business center printing fax services available 100 nonsmoking pet free rooms air conditioned fans available selfparking available virginia mason hospital fee
Name: The Baroness Hot

In [29]:
df.set_index('name', inplace = True)

##Máquina Preditiva com Machine Learning

In [35]:

#ngram_range(1,3) significa que vamos pegar unigramas, bigramas e trigamas
tf = TfidfVectorizer(analyzer = 'word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['desc_clean'])
cosine_similarities = linear_kernel(tFidf_matrix, tfidf_matrix)

In [36]:
indices = pd.Series(df.index)

In [38]:
indices[:50]

0                        Hilton Garden Seattle Downtown
1                                Sheraton Grand Seattle
2                         Crowne Plaza Seattle Downtown
3                         Kimpton Hotel Monaco Seattle 
4                                    The Westin Seattle
5                           The Paramount Hotel Seattle
6                                        Hilton Seattle
7                                         Motif Seattle
8                                       Warwick Seattle
9                            Four Seasons Hotel Seattle
10                                            W Seattle
11                                   Gand Hyatt Seattle
12                                 Kimpton Alexis Hotel
13                                            Hotel Max
14                                    Ace Hotel Seattle
15                          Seattle Marriott Waterfront
16                          The Edgewater Hotel Seattle
17                   SpringHill Suites Seattle D

In [46]:
def recommendations(name, cosine_similarities = cosine_similarities):
    
    recommended_hotels = []

    #obtendo o indice do hotel que corresponde ao nome
    idx = indices[indices == name].index[0]

    #criando uma serie com as pontuacoes de similaridade em ordem descrescente
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

    #obtendo os indices dos 10 hoteis mais semelhantes, exceto ele proprio
    top_10_indexes = list(score_series.iloc[1:11].index)

    #preencher a lista com os nomes dos 10 principais hoteis correspondentes
    for i in top_10_indexes:
        recommended_hotels.append(list(df.index)[i])

    return recommended_hotels

In [47]:
recommendations('Hilton Seattle Airport & Conference Center')

['Embassy Suites by Hilton Seattle Tacoma International Airport',
 'DoubleTree by Hilton Hotel Seattle Airport',
 'Seattle Airport Marriott',
 'Motel 6 Seattle Sea-Tac Airport South',
 'Econo Lodge SeaTac Airport North',
 'Four Points by Sheraton Downtown Seattle Center',
 'Knights Inn Tukwila',
 'Econo Lodge Renton-Bellevue',
 'Hampton Inn Seattle/Southcenter',
 'Radisson Hotel Seattle Airport']

In [48]:
recommendations('The Bacon Mansion Bed and Breakfast')

['11th Avenue Inn Bed and Breakfast',
 'Shafer Baillie Mansion Bed & Breakfast',
 'Chittenden House Bed and Breakfast',
 'Gaslight Inn',
 'Bed and Breakfast Inn Seattle',
 'Silver Cloud Hotel - Seattle Broadway',
 'Hyatt House Seattle',
 'Mozart Guest House',
 'Quality Inn & Suites Seattle Center',
 'MarQueen Hotel']